In [1]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
import psycopg2

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5433',
                host = 'localhost',                
                database = 'inicudb')

cur  = con.cursor()

In [2]:
uhidPatient = "RNEH.0000008375"
caseType = 'Death'
conditionCase = "(dischargestatus = 'Death' or dischargestatus = 'LAMA' )"
#uhidPatient = "RSHI.0000012984"
#caseType = 'Discharge'
#conditionCase = "(dischargestatus = 'Discharge')"
seperator = '_'

In [3]:
#base = pd.read_csv("/Users/harpreet/Videos/data_22_june.csv",  nrows=999999)

In [4]:
#ds = base[~base.starttime.notna()]

In [5]:
#ds.hour_series.head()

In [6]:
#base[~base.hour_series.notna()]

In [7]:
#death_base = base[base.dischargestatus == 1]

In [8]:
#discharge_base = base[base.dischargestatus == 0]

In [9]:
#death_base.uhid.unique()


In [10]:
#discharge_base.uhid.unique()

In [11]:
#dea = death_base.groupby('uhid').head(1)
#dis = discharge_base.groupby('uhid').head(1)

In [12]:
#checkingUHIDData = death_base[death_base.uhid==uhidPatient]

In [13]:
#checkingUHIDData.to_csv(uhidPatient+".csv")

In [14]:
#print(dea.gestationweekbylmp, dea.birthweight,dea.uhid)

In [15]:
#print(dea.gestationweekbylmp, dea.birthweight,dea.uhid)

In [16]:
#lesstha26 = dis[(dis.gestationweekbylmp <= 26) & (dis.birthweight<=1500)]
#between26and28 = dis[(dis.gestationweekbylmp > 26) & (dis.gestationweekbylmp <= 28) & (dis.birthweight<=1500)]
#between28and32 = dis[(dis.gestationweekbylmp > 28) & (dis.gestationweekbylmp <= 32) & (dis.birthweight<=1500)]
#greaterthan32 = dis[(dis.gestationweekbylmp > 32) & (dis.birthweight<=3000) & (dis.birthweight>=2500)]

In [17]:
#(lesstha26.birthweight)




In [18]:
#(between26and28.birthweight)

In [19]:
#(between28and32.birthweight)

In [20]:
#len(greaterthan32)

In [21]:
cur10 = con.cursor()
cur10.execute("SELECT t1.uhid,t1.dateofbirth,t1.timeofbirth,t1.timeofadmission,t1.dateofadmission,t1.gender,t1.birthweight,t1.birthlength,t1.birthheadcircumference,t1.inout_patient_status,t1.gestationweekbylmp,t1.gestationdaysbylmp,t1.baby_type,t1.central_temp,t1.dischargeddate,t2.apgar_onemin,t2.apgar_fivemin,t2.apgar_tenmin,t3.motherage,t4.conception_type,t4.mode_of_delivery,t5.steroidname,t5.numberofdose,t1.dischargestatus  FROM apollo.baby_detail AS t1 LEFT JOIN apollo.birth_to_nicu AS t2 ON t1.uhid=t2.uhid LEFT JOIN apollo.parent_detail AS t3 ON t1.uhid=t3.uhid LEFT JOIN apollo.antenatal_history_detail AS t4 ON t1.uhid=t4.uhid LEFT JOIN apollo.antenatal_steroid_detail AS t5 ON t1.uhid=t5.uhid where timeofadmission is not null and timeofbirth is not null and t1.uhid = '"+uhidPatient+"';")
cols10 = list(map(lambda x: x[0], cur10.description))
fixed = pd.DataFrame(cur10.fetchall(),columns=cols10)

In [22]:
len(fixed)

1

In [23]:
s1 = set(fixed.uhid.unique())

In [24]:
cur11 = con.cursor()
cur11.execute("SELECT DISTINCT(uhid) FROM apollo.baby_detail WHERE dateofadmission >= '2018-07-01' AND dateofadmission <= '2020-06-25' and UHID IN  (select distinct(uhid) from apollo.babyfeed_detail where uhid in  ( select distinct(uhid) from apollo.baby_visit where uhid in (select  distinct(uhid) from apollo.nursing_vitalparameters where uhid in ( select distinct(uhid) from apollo.device_monitor_detail UNION select distinct(uhid) from apollo.device_monitor_detail_dump)))) and "+conditionCase+" and isreadmitted is not true and gestationweekbylmp is not null and birthweight is not null and uhid = '"+uhidPatient+"';")
cols11 = list(map(lambda x: x[0], cur11.description))
uhids = pd.DataFrame(cur11.fetchall(),columns=cols11)

In [25]:
print(len(uhids.uhid.unique()))

1


In [26]:
s2 = set(uhids.uhid.unique())

In [27]:
l = list(s1 - s1.intersection(s2))

In [28]:
len(uhids)

1

In [29]:
fixed.set_index('uhid',inplace=True)

In [30]:
df = fixed.drop(l)

In [31]:
len(df.drop_duplicates())

1

In [32]:
df.reset_index(inplace=True)

In [33]:
len(df.uhid.unique())

1

In [34]:
#dates_detail = ds.copy()

In [35]:
dates_detail = df.copy()

In [36]:
dates_detail.head()

,uhid,dateofbirth,timeofbirth,timeofadmission,dateofadmission,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,dischargeddate,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus
0,RNEH.0000008375,2018-09-12,"08,38,AM","09,00,AM",2018-09-12,Female,845.0,None,None,In Born,...,2018-09-25 04:08:17.263,None,None,None,None,ivf,LSCS,Beta,2,Death


In [37]:
def second_addition(x):
    try:
        if x.split(",")[2] == 'PM':
            if x.split(",")[0] != '12':
                return (12 + float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
            else:
                return (float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
        else:
            return (float(x.split(",")[0]) + float(x.split(",")[1])/60)*3600
    except:
        
        if x.split(" ")[1] == 'PM':
            if (x.split(" ")[0]).split(":")[0] != '12':
                return (12 + float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
            else:
                return (float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
        else:
            return (float((x.split(" ")[0]).split(":")[0]) + float((x.split(" ")[0]).split(":")[1])/60)*3600
    
        

In [38]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [39]:
dates_detail['add_seconds'] = dates_detail['timeofbirth'].apply(second_addition)
dates_detail['add_seconds_admission'] = dates_detail['timeofadmission'].apply(second_addition)

In [40]:
from datetime import timedelta

In [41]:
def actual_birthdate(x,y):
    try:
        return pd.to_datetime(x) + timedelta(seconds=y)
    except:
        pass

In [42]:
dates_detail['actual_DOB'] = dates_detail.apply(lambda x: actual_birthdate(x['dateofbirth'], x['add_seconds']), axis=1)

In [43]:
dates_detail['actual_DOA'] = dates_detail.apply(lambda x: actual_birthdate(x['dateofadmission'], x['add_seconds_admission']), axis=1)

In [44]:
dd = dates_detail[['actual_DOB','uhid','dischargeddate','actual_DOA','gender','birthweight','birthlength','birthheadcircumference','inout_patient_status'
,'gestationweekbylmp','gestationdaysbylmp','baby_type','central_temp','apgar_onemin',
'apgar_fivemin','apgar_tenmin','motherage','conception_type','mode_of_delivery','steroidname',
'numberofdose','dischargestatus','birthlength','birthheadcircumference','inout_patient_status'
,'gestationweekbylmp','gestationdaysbylmp','baby_type','central_temp','apgar_onemin',
'apgar_fivemin','apgar_tenmin','motherage','conception_type','mode_of_delivery','steroidname',
'numberofdose']]

In [45]:
dd.dropna(subset=['dischargeddate'],inplace=True)

/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
dd['los'] = dates_detail['dischargeddate'] - dates_detail['actual_DOA']

/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
def day(x):
    return x.total_seconds()/86400

In [48]:
dd['day_1'] = dd['los'].apply(day)

/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
dd = dd[dd['day_1']>=0]

In [50]:
dd.sort_values(by = ['actual_DOA'],inplace=True)

In [51]:
dd.drop_duplicates(subset=['uhid'],keep='first',inplace=True)

In [52]:
dd

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422


In [53]:
dt = pd.DataFrame()

In [54]:
dt

""


In [55]:
dd

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422


In [56]:
dd.columns

Index(['actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA', 'gender',
       'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength',
       'birthheadcircumference', 'inout_patient_status', 'gestationweekbylmp',
       'gestationdaysbylmp', 'baby_type', 'central_temp', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'los', 'day_1'],
      dtype='object')

In [57]:
dd.dischargeddate

0   2018-09-25 04:08:17.263
Name: dischargeddate, dtype: datetime64[ns]

In [58]:
dd.dischargeddate.iloc[0]

Timestamp('2018-09-25 04:08:17.263000')

In [59]:
dd.actual_DOA.iloc[0]

Timestamp('2018-09-12 09:00:00')

In [60]:
x = dd[dd['uhid']==uhidPatient]

In [61]:
x

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422


In [62]:
(x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()

1105697.263

In [63]:
import math
for i in dd.uhid.unique():
    x = dd[dd['uhid']==i]
    time_s = []
    n = math.ceil((x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()/60)
    try:
        x = pd.concat([x]*int(n))

        for i in range(0,len(x)):
            #print(i)
            time_s.append(x['actual_DOA'].iloc[i] + timedelta(seconds = i*60))
            #t['hour_series'].loc[i] = t['actual_DOA'].iloc[i] + timedelta(hours = i)

        x['hour_series'] = time_s
    except Exception as e:
        print(i,n, e)
        continue
    dt = dt.append(x,ignore_index=True)


In [64]:
len(dt)

18429

In [65]:
dt

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00


In [66]:
dt

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00


In [67]:
dt.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [68]:
dt

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00


In [69]:
cur1 = con.cursor()
cur1.execute("select distinct(b.uhid),l.conception_type, b.gender, b.dateofadmission, b.dischargestatus,b.birthweight,b.weight_galevel,b.weight_centile,b.birthlength,b.birthheadcircumference, b.inout_patient_status, b.gestationweekbylmp, b.gestationdaysbylmp,round( CAST((b.gestationweekbylmp + b.gestationdaysbylmp/7::float) as numeric),2) as Gestation,b.dischargeddate, b.admissionweight, b.baby_type,b.baby_number, b.branchname ,DATE_PART('day',b.dischargeddate - b.dateofadmission) as LOS,c.apgar_onemin, c.apgar_fivemin,c.apgar_tenmin, c.resuscitation,d.isantenatalsteroidgiven, d.mode_of_delivery,z.motherage,e.jaundicestatus as JAUNDICE,  f.eventstatus as SEPSIS,f.progressnotes, g.eventstatus as RDS, g.progressnotes,y.eventstatus as ASPHYXIA,y.progressnotes from apollo.baby_detail as b left join apollo.birth_to_nicu as c on b.uhid = c.uhid and b.episodeid = c.episodeid left join apollo.parent_detail as z on b.uhid = z.uhid and b.episodeid = z.episodeid left join apollo.antenatal_history_detail as d on b.uhid = d.uhid and b.episodeid = d.episodeid left join apollo.sa_jaundice AS e ON b.uhid = e.uhid and e.jaundicestatus = 'Yes' and e.phototherapyvalue='Start' left join apollo.sa_infection_sepsis AS f ON b.uhid = f.uhid and f.eventstatus = 'yes' and f.episode_number = 1 left join apollo.sa_cns_asphyxia AS y ON b.uhid = y.uhid and y.eventstatus = 'yes' and y.episode_number = 1 left join apollo.antenatal_history_detail as l on b.uhid=l.uhid left join apollo.sa_resp_rds AS g ON b.uhid = g.uhid and g.eventstatus = 'Yes' and g.episode_number = 1 and g.uhid IN (select distinct(h.uhid) from apollo.respsupport AS h where h.eventname='Respiratory Distress' and (h.rs_vent_type ='Mechanical Ventilation' OR h.rs_vent_type ='HFO') UNION select distinct(i.uhid) from apollo.sa_resp_rds AS i where i.sufactantname is not null) and b.uhid = '"+uhidPatient+ "' order by b.dateofadmission")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
#ds.sort_values('modificationtime',inplace=True)

In [70]:
len(dt.uhid.unique())

1

In [71]:
data = ds.copy()

In [72]:
dt

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,...,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,...,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00


In [73]:
ids = dt.uhid.unique()

In [74]:
len(dt)

18429

In [75]:
data.drop_duplicates('uhid',keep='first',inplace=True)

In [76]:
df = pd.DataFrame(columns=data.columns)

In [77]:
for i in ids:
    x = data[data['uhid']==i]
    df = df.append(x,ignore_index=True)

In [78]:
#d = pd.read_csv('baseline.csv')

In [79]:
#len(d)

In [80]:
#s = set(d['uhid'].unique())

In [81]:
pd.set_option('display.max_columns',100)

In [82]:
#s1 = set(ids)

In [83]:
ds = pd.DataFrame()

In [84]:
ds = dt.copy()

In [85]:
#ds.to_csv('test_baseline.csv')

In [86]:
uhid = ds.uhid.unique()

In [87]:
dt = pd.DataFrame()

In [88]:
for i in uhid:
    x = dd[dd['uhid']==i]
    time_s = []
    n = math.ceil((x.dischargeddate.iloc[0] - x.actual_DOA.iloc[0]).total_seconds()/60)
    try:
        x = pd.concat([x]*int(n))
        for i in range(0,len(x)):
            #print(i)
            time_s.append(x['actual_DOA'].iloc[i] + timedelta(seconds = i*60))
            #t['hour_series'].loc[i] = t['actual_DOA'].iloc[i] + timedelta(hours = i)

        x['hour_series'] = time_s
    except Exception as inst:
        print(inst)
        continue
    dt = dt.append(x,ignore_index=True)


In [89]:
dd['uhid']

0    RNEH.0000008375
Name: uhid, dtype: object

In [90]:
len(dt)

18429

In [91]:
x

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00


In [92]:
uhid

array(['RNEH.0000008375'], dtype=object)

In [93]:
dt['hour_series'].apply(split_hour)

0        2018-09-12 09
1        2018-09-12 09
2        2018-09-12 09
3        2018-09-12 09
4        2018-09-12 09
             ...      
18424    2018-09-25 04
18425    2018-09-25 04
18426    2018-09-25 04
18427    2018-09-25 04
18428    2018-09-25 04
Name: hour_series, Length: 18429, dtype: object

In [94]:
dt['hour'] = dt['hour_series'].apply(split_hour)

In [95]:
dt['day'] = dt['hour_series'].apply(split_date_1)

In [96]:
len(dt.uhid.unique())

1

In [97]:
#output
cur9= con.cursor()
cur9.execute("SELECT t1.uhid,t1.abdomen_girth,t1.urine,t1.stool,t1.stool_passed, t1.entry_timestamp FROM apollo.nursing_intake_output AS t1 where t1.uhid = '"+uhidPatient+"'")
cols9 = list(map(lambda x: x[0], cur9.description))
output = pd.DataFrame(cur9.fetchall(),columns=cols9)

In [98]:
dt

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,2018-09-25


In [99]:
output_d = pd.DataFrame()

In [100]:
for i in uhid:
    x = output[output['uhid']==i]
    output_d = output_d.append(x,ignore_index=True)

In [101]:
output_d.entry_timestamp = output_d.entry_timestamp + timedelta(seconds=19800)

In [102]:
output_d

,uhid,abdomen_girth,urine,stool,stool_passed,entry_timestamp
0,RNEH.0000008375,None,25,Medium,True,2018-09-13 18:00:41.327000+08:00
1,RNEH.0000008375,None,None,None,None,2018-09-14 00:06:32.578000+08:00
2,RNEH.0000008375,None,25,None,None,2018-09-13 20:00:33.735000+08:00
3,RNEH.0000008375,None,None,None,None,2018-09-15 11:47:19.171000+08:00
4,RNEH.0000008375,None,20,Small,True,2018-09-15 12:12:12.842000+08:00
5,RNEH.0000008375,None,25,Medium,True,2018-09-15 11:46:45.055000+08:00
6,RNEH.0000008375,None,None,None,None,2018-09-15 11:47:26.093000+08:00
7,RNEH.0000008375,None,None,None,None,2018-09-19 22:50:55.928000+08:00
8,RNEH.0000008375,None,None,None,None,2018-09-19 22:51:54.532000+08:00
9,RNEH.0000008375,None,None,None,None,2018-09-21 08:10:15.825000+08:00


In [103]:
output_d['hour'] = output_d.entry_timestamp.apply(split_hour)
output_d['day'] = output_d.entry_timestamp.apply(split_date_1)

In [104]:
output_d

,uhid,abdomen_girth,urine,stool,stool_passed,entry_timestamp,hour,day
0,RNEH.0000008375,None,25,Medium,True,2018-09-13 18:00:41.327000+08:00,2018-09-13 18,2018-09-13
1,RNEH.0000008375,None,None,None,None,2018-09-14 00:06:32.578000+08:00,2018-09-14 00,2018-09-14
2,RNEH.0000008375,None,25,None,None,2018-09-13 20:00:33.735000+08:00,2018-09-13 20,2018-09-13
3,RNEH.0000008375,None,None,None,None,2018-09-15 11:47:19.171000+08:00,2018-09-15 11,2018-09-15
4,RNEH.0000008375,None,20,Small,True,2018-09-15 12:12:12.842000+08:00,2018-09-15 12,2018-09-15
5,RNEH.0000008375,None,25,Medium,True,2018-09-15 11:46:45.055000+08:00,2018-09-15 11,2018-09-15
6,RNEH.0000008375,None,None,None,None,2018-09-15 11:47:26.093000+08:00,2018-09-15 11,2018-09-15
7,RNEH.0000008375,None,None,None,None,2018-09-19 22:50:55.928000+08:00,2018-09-19 22,2018-09-19
8,RNEH.0000008375,None,None,None,None,2018-09-19 22:51:54.532000+08:00,2018-09-19 22,2018-09-19
9,RNEH.0000008375,None,None,None,None,2018-09-21 08:10:15.825000+08:00,2018-09-21 08,2018-09-21


In [105]:
output_d.sort_values(by=['uhid','entry_timestamp'],inplace=True)

In [106]:
len(output_d)

16

In [107]:
test = output_d.drop_duplicates(subset=['uhid','hour'],keep='last')

In [108]:
test.head()

,uhid,abdomen_girth,urine,stool,stool_passed,entry_timestamp,hour,day
0,RNEH.0000008375,None,25,Medium,True,2018-09-13 18:00:41.327000+08:00,2018-09-13 18,2018-09-13
2,RNEH.0000008375,None,25,None,None,2018-09-13 20:00:33.735000+08:00,2018-09-13 20,2018-09-13
1,RNEH.0000008375,None,None,None,None,2018-09-14 00:06:32.578000+08:00,2018-09-14 00,2018-09-14
6,RNEH.0000008375,None,None,None,None,2018-09-15 11:47:26.093000+08:00,2018-09-15 11,2018-09-15
4,RNEH.0000008375,None,20,Small,True,2018-09-15 12:12:12.842000+08:00,2018-09-15 12,2018-09-15


In [109]:
dt.head()

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12


In [110]:
dh = pd.merge(dt,test,on=['uhid','hour'],how='left')

In [111]:
dh

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN


In [112]:
#dh.sort_values(by=['uhid','entry_timestamp'],inplace=True)

In [113]:
dh.sort_values('hour_series')

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN


In [114]:
dg = pd.DataFrame()

In [115]:
dh

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN


In [116]:
len(dh)

18429

In [117]:
x.columns

Index(['uhid', 'abdomen_girth', 'urine', 'stool', 'stool_passed',
       'entry_timestamp'],
      dtype='object')

In [118]:
x.urine

0       25
1     None
2       25
3     None
4       20
5       25
6     None
7     None
8     None
9     None
10      24
11    None
12      20
13    None
14    None
15    None
Name: urine, dtype: object

In [119]:
for i in uhid:
    x = dh[dh['uhid']==i]
    #x.dropna(subset=['urine'],inplace=True)
    x['time_divide'] = 0
    print(len(x))
    startTime = None
    for j in range(len(x)):
        try:
            if j == 0:
                x.loc[j,'time_divide'] = 0.0                
                startTime = x.actual_DOA.iloc[j]
            else:
                if(x.urine.iloc[j] != None):
                    urine_value = float(x.urine.iloc[j])
                    if(~math.isnan(urine_value) and (urine_value >0)):
                        x.loc[j,'time_divide'] = (pd.to_datetime(str(x.entry_timestamp.iloc[j]).split("+")[0]) - startTime).total_seconds()
                        startTime = pd.to_datetime(str(x.entry_timestamp.iloc[j]).split("+")[0])
            #print(x.loc[j,'time_divide'])
        except Exception as e:
            print(j,'Exception is',e)
            continue
    dg = dg.append(x,ignore_index=True)    

18429


In [120]:
dg.time_divide.unique()

array([     0.   , 118841.327,   7192.408, 144699.107, 474469.605,
        64828.19 ])

In [121]:
dg

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y,time_divide
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,201

In [122]:
dg = dg.sort_values('hour_series')

In [123]:
dg

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y,time_divide
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,201

In [124]:
dg.time_divide.unique()

array([     0.   , 118841.327,   7192.408, 144699.107, 474469.605,
        64828.19 ])

In [125]:
dg = dg.sort_values('hour_series')

In [126]:
dg

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y,time_divide
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,201

In [127]:
dg.columns


Index(['actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA', 'gender',
       'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength',
       'birthheadcircumference', 'inout_patient_status', 'gestationweekbylmp',
       'gestationdaysbylmp', 'baby_type', 'central_temp', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'los', 'day_1',
       'hour_series', 'hour', 'day_x', 'abdomen_girth', 'urine', 'stool',
       'stool_passed', 'entry_timestamp', 'day_y', 'time_divide'],
      dtype='object')

In [128]:
dg.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_1.csv')

In [129]:
dg

,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,dischargestatus,birthlength,birthheadcircumference,inout_patient_status,gestationweekbylmp,gestationdaysbylmp,baby_type,central_temp,apgar_onemin,apgar_fivemin,apgar_tenmin,motherage,conception_type,mode_of_delivery,steroidname,numberofdose,los,day_1,hour_series,hour,day_x,abdomen_girth,urine,stool,stool_passed,entry_timestamp,day_y,time_divide
0,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:00:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
1,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:01:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
2,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:02:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
3,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:03:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
4,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-12 09:04:00,2018-09-12 09,2018-09-12,NaN,NaN,NaN,NaN,NaT,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18424,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:04:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18425,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:05:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18426,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:06:00,2018-09-25 04,2018-09-25,NaN,NaN,NaN,NaN,NaT,NaN,0.0
18427,2018-09-12 08:38:00,RNEH.0000008375,2018-09-25 04:08:17.263,2018-09-12 09:00:00,Female,845.0,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,Death,None,None,In Born,28,5,Triplets,36.5,None,None,None,None,ivf,LSCS,Beta,2,12 days 19:08:17.263000,12.797422,2018-09-25 04:07:00,2018-09-25 04,201